---

## Segmenting and Clustering Neighborhoods in Toronto (Week 3 Assignment)

---



In [2]:
# Import libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests

print("Libraries imported successfully")

Libraries imported successfully



#### Scrape the data using BeautifulSoup
---

In [7]:
# Get the xml from the source
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
source = requests.get(url).text
soup = BeautifulSoup(source, 'xml')

In [10]:
# Extract the table from the Wikipedia page
table = soup.find('table',{'class':'wikitable sortable'})

In [11]:
# Fetch the data from the table
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    td=[]
    for t in row.find_all('td'):
        td.append(t.text.strip())
    data.append(td)

In [15]:
# Put the data in a Dataframe
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


##### Data Cleaning
---

In [16]:
# Remove rows where Borough is "Not assigned"
df=df[df['Borough']!='Not assigned']

In [22]:
# If a cell has a borough but Neigborhood = "Not assigned" , then the neighborhood = borough.
df['Neighborhood'] = df.apply(
    lambda row: 
    row['Borough'] if row['Neighborhood'] == 'Not assigned' 
    else row['Neighborhood'],
    axis=1)

In [23]:
# More than one neighborhood can exist in one postal code area. .  
# The rows from the neighborhoods will be combined into one row with the neighborhoods separated with a comma.
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].\
    apply(', '.join).to_frame()
df.reset_index(inplace=True)

In [24]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [25]:
df.shape

(103, 3)